In [74]:
# Proceso ETL para cargar archivos CSV a MongoDB

import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import nltk
from nltk.corpus import stopwords


In [61]:
# Descargar las stopwords en español
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
# Configuración de MongoDB
MONGO_URI = "mongodb://localhost:27017/"  # Cambia esto si usas una configuración diferente
DATABASE_NAME = "tesis"
COLLECTION_NAME = "ciberbullying"


In [63]:
# Lista de archivos CSV
csv_files = [
    "dataset_transformed1.csv",
    "dataset_transformed2.csv",
    "dataset_transformed3.csv",
    "dataset_transformed4.csv",
    "dataset_transformed5.csv"
]

In [64]:
def load_csv_to_mongo(file_list, db_name, collection_name):
    """
    Carga múltiples archivos CSV a una colección de MongoDB.

    Parameters:
        file_list (list): Lista de rutas de archivos CSV.
        db_name (str): Nombre de la base de datos en MongoDB.
        collection_name (str): Nombre de la colección en MongoDB.
    """
    try:
        # Conexión a MongoDB
        client = MongoClient(MONGO_URI)
        db = client[db_name]
        collection = db[collection_name]

        for file in file_list:
            print(f"Procesando archivo: {file}")
            
            # Leer el archivo CSV
            df = pd.read_csv(file)
            
            # Limpiar y convertir a JSON
            data = df.to_dict(orient='records')

            # Insertar datos en MongoDB
            if data:
                collection.insert_many(data)
                print(f"Datos de {file} cargados correctamente.")
            else:
                print(f"El archivo {file} está vacío o no contiene datos válidos.")
        
        print("Todos los archivos se han cargado exitosamente.")
    except Exception as e:
        print(f"Error durante el proceso ETL: {e}")
    finally:
        client.close()

In [65]:
# Ejecutar el ETL
load_csv_to_mongo(csv_files, DATABASE_NAME, COLLECTION_NAME)

Procesando archivo: dataset_transformed1.csv
Datos de dataset_transformed1.csv cargados correctamente.
Procesando archivo: dataset_transformed2.csv
Datos de dataset_transformed2.csv cargados correctamente.
Procesando archivo: dataset_transformed3.csv
Datos de dataset_transformed3.csv cargados correctamente.
Procesando archivo: dataset_transformed4.csv
Datos de dataset_transformed4.csv cargados correctamente.
Procesando archivo: dataset_transformed5.csv
Datos de dataset_transformed5.csv cargados correctamente.
Todos los archivos se han cargado exitosamente.


In [66]:
# --- Análisis de Procesamiento de Lenguaje Natural (PLN) ---

# Cargar datos desde MongoDB
def fetch_data_from_mongo(db_name, collection_name):
    """
    Extrae datos de una colección MongoDB y los convierte en un DataFrame.
    """
    client = MongoClient(MONGO_URI)
    db = client[db_name]
    collection = db[collection_name]
    data = pd.DataFrame(list(collection.find()))
    client.close()
    return data

data = fetch_data_from_mongo(DATABASE_NAME, COLLECTION_NAME)

In [67]:
# Limpieza de datos
def clean_data(data, text_column):
    """
    Limpia el DataFrame eliminando o reemplazando valores nulos en la columna de texto.

    Parameters:
        data (DataFrame): Datos con texto y etiquetas.
        text_column (str): Nombre de la columna de texto.

    Returns:
        DataFrame: Datos limpios.
    """
    data[text_column] = data[text_column].fillna("")  # Reemplazar NaN con cadena vacía
    return data

# Limpieza de los datos antes del preprocesamiento
data = clean_data(data, "texto")

In [68]:
# Preprocesamiento del texto
def preprocess_text(data, text_column, label_columns):
    """
    Preprocesa los textos y divide en conjuntos de entrenamiento y prueba.

    Parameters:
        data (DataFrame): Datos con columnas de texto y etiquetas.
        text_column (str): Nombre de la columna con texto.
        label_columns (list): Nombres de las columnas de etiquetas.

    Returns:
        X_train, X_test, y_train, y_test: Conjuntos de datos divididos.
    """
    vectorizer = TfidfVectorizer(stop_words=spanish_stopwords)
    X = vectorizer.fit_transform(data[text_column])
    y = data[label_columns]
    return train_test_split(X, y, test_size=0.2, random_state=42)

text_column = "texto"  # Columna de texto
label_columns = [
    "insulto", "amenaza", "rumores_difamatorios", "comentarios_humillantes", "lenguaje_discriminatorio",
    "acoso_sexual", "manipulacion_emocional", "lenguaje_vulgar", "acoso_general", "redes_sociales",
    "suicidio_autolesion"
]  # Columnas de etiquetas

In [69]:
# Preprocesamiento
X_train, X_test, y_train, y_test = preprocess_text(data, text_column, label_columns)